# Creating and Manipulating Tensors

**Learning Objectives:**
  * Initialize and assign TensorFlow `Variable`s
  * Create and manipulate tensors.
  * Refresh your memory about addition and multiplication in linear algebra (consult an introduction to matrix [addition](https://en.wikipedia.org/wiki/Matrix_addition) and [multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication) if this topic is new to you).
  * Familiarize yourself with basic TensorFlow math and array operations

In [ ]:
import tensorflow as tf

## Vector Addition

You can perform many typical mathematical operations on tensors ([TF API](https://www.tensorflow.org/api_guides/python/math_ops)). The following code
cell creates and manipulates vectors (1-D tensors) all having exactly six elements:

In [ ]:
with tf.Graph().as_default():
  # Create a six-element vector (1-D tensor).
  primes = tf.constant([2, 3, 5, 7, 11, 13], dtype=tf.int32)

  # Create another six-element vector. Each element in the vector will be
  # initialized to 1. The first argument is the shape of the tensor (more
  # on shapes below).
  ones = tf.ones([6], dtype=tf.int32)

  # Add the two tensors. The resulting tensor is a six-element vector.
  just_beyond_primes = tf.add(primes, ones)

  # Create a session to run the default graph.
  with tf.Session() as sess:
    print just_beyond_primes.eval()

### Tensor Shapes

Shapes are used to characterize the size and number of dimensions of a tensor. The shape of a tensor is expressed as `list`, with the `i`th element representing the size along dimension `i`. The length of the list then indicates the rank of the tensor (i.e., the number of dimensions).

For more information, see the [TensorFlow documentation](https://www.tensorflow.org/programmers_guide/dims_types).

A few basic examples:

In [ ]:
with tf.Graph().as_default():
  # A scalar is a 0-D tensor.
  scalar = tf.zeros([])

  # A vector with 3 elements.
  vector = tf.zeros([3])

  # A matrix with 2 rows and 3 columns.
  matrix = tf.zeros([2, 3])

  with tf.Session() as sess:
    print 'scalar has shape', scalar.get_shape(), 'and value:\n', scalar.eval()
    print 'vector has shape', vector.get_shape(), 'and value:\n', vector.eval()
    print 'matrix has shape', matrix.get_shape(), 'and value:\n', matrix.eval()


### Broadcasting

In mathematics, you may only perform element-wise operations (e.g. *add* and *equals*) on tensors of the same shape.  In TensorFlow, however, you may perform operations on tensors that would traditionally have been incompatible. **Broadcasting** -- a concept borrowed from numpy -- enables TensorFlow to automatically enlarge the smaller array in an operation to be the shape compatible for the element-wise operation.

When a tensor gets broadcast, its entries along that dimension get conceptually **copied**. (They are not actually copied for performance reasons. Broadcasting was invented as a performance optimization.)

For instance, broadcasting enables the following:

* If an operand requires a size `[6]` tensor, a size `[1]` or a size `[]` tensor can serve as an operand.
* If an operation requires a size `[4, 6]` tensor, any of the following sizes can serve as an operand:
  * `[1, 6]`
  * `[6]`
  * `[]`
* If an operation requires a size `[3, 5, 6]` tensor, any of the following sizes can serve as an operand:

  * `[1, 5, 6]`
  * `[3, 1, 6]`
  * `[3, 5, 1]`
  * `[1, 1, 1]`
  * `[5, 6]`
  * `[1, 6]`
  * `[6]`
  * `[1]`
  * `[]`

The full ruleset is well described by the easy-to-read [numpy broadcasting documentation](http://docs.scipy.org/doc/numpy-1.10.1/user/basics.broadcasting.html).

The following code performs the same tensor addition as before, but using broadcasting:

In [ ]:
with tf.Graph().as_default():
  # Create a six-element vector (1-D tensor).
  primes = tf.constant([2, 3, 5, 7, 11, 13], dtype=tf.int32)

  # Create a constant scalar with value 1.
  ones = tf.constant(1, dtype=tf.int32)

  # Add the two tensors. The resulting tensor is a six-element vector.
  just_beyond_primes = tf.add(primes, ones)

  with tf.Session() as sess:
    print just_beyond_primes.eval()

## Matrix Multiplication

In linear algebra, when multiplying two matrices, the number of *columns* of the first matrix must
equal the number of *rows* in the second matrix.

- It is valid to multiply a `3x4` matrix with a `4x2` matrix. This will result in a `3x2` matrix.
- It is invalid to multiply a `4x2` matrix with a `3x4` matrix.

In [ ]:
with tf.Graph().as_default():
  # Create a matrix (2-d tensor) with 3 rows and 4 columns.
  x = tf.constant([[5, 2, 4, 3], [5, 1, 6, -2], [-1, 3, -1, -2]],
                  dtype=tf.int32)

  # Create a matrix with 4 rows and 2 columns.
  y = tf.constant([[2, 2], [3, 5], [4, 5], [1, 6]], dtype=tf.int32)

  # Matrix multiply the two operands. 
  # The resulting matrix will have 3 rows and 2 columns.
  matrix_multiply_result = tf.matmul(x, y)

  with tf.Session() as sess:
    print matrix_multiply_result.eval()

## Tensor Reshaping

With tensor addition and matrix multiplication each imposing significant constraints
on operands, TensorFlow programmers must frequently reshape tensors. Use
the `tf.reshape` method to reshape a tensor. 
For example, you could reshape a 20x2 tensor into a 2x20 tensor or a 10x4 tensor.
You can also use `tf.reshape` to change the number of dimensions (the "rank") of the tensor.
For example, you could reshape that 20x2 tensor into a 1-D 40-element tensor or a 3-D 5x2x4 tensor.

In [ ]:
with tf.Graph().as_default():
  # Create a 4-element vector (1-D tensor).
  a = tf.constant([5, 2, 4, 3], dtype=tf.int32)

  # Reshape that 4-element vector into a 2x2 matrix.
  reshaped_matrix = tf.reshape(a, [2,2])

  with tf.Session() as sess:
    print reshaped_matrix.eval()

### Exercise #1: Reshape two tensors in order to multiply them.

The following two vectors are incompatible for matrix multiplication:

  *  `a = tf.constant([5, 3, 2, 7, 1, 4])`
  *  `b = tf.constant([4, 6, 3])`

Reshape these vectors into compatible operands for matrix multiplication.
Then, invoke a matrix multiplication operation on the reshaped tensors.

In [ ]:
  # Write your code for Task 1 here.

## Variables, Initialization and Assignment

So far, all the concepts we've shown were stateless; calling `eval()` always returned the same result. TensorFlow `Variable` objects allow you to keep and change state. When creating a variable, you may use an initial value, or you may use an initializer (like a distribution).

In [ ]:
g = tf.Graph()
with g.as_default():
  # Create a variable with the initial value 3
  v = tf.Variable([3])

  # Create a variable of shape [1], with a random initial value,
  # sampled from a normal distribution with mean 1 and standard deviation 0.35
  w = tf.Variable(tf.random_normal([1], mean=1.0, stddev=0.35))

One particularity of TensorFlow is that **variable initialization is not automatic**. For example, the following block will cause an error:

In [ ]:
with g.as_default():
  with tf.Session() as sess:
    try:
      v.eval()
    except tf.errors.FailedPreconditionError as e:
      print "Caught expected error: ", e

The easiest way to initialize a variable is to call `global_variables_initializer`. Note the use of `Session.run()`, which is roughly equivalent to `eval()`.

In [ ]:
with g.as_default():
  with tf.Session() as sess:
    initialization = tf.global_variables_initializer()
    sess.run(initialization)
    # Now, variables can be accessed normally, and they have values assigned to
    # them.
    print v.eval()
    print w.eval()


Once initialized, variables will maintain their value within the same session. However, when starting a new session, you will need to re-initialize them:

In [ ]:
with g.as_default():
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # These three prints will print the same value.
    print w.eval()
    print w.eval()
    print w.eval()

To change the value of a variable, use the `assign` op. Note that, simply creating the `assign` op will not have any effect. Like with initialization, you have to `run` the assignment op before it has the desired effect.

In [ ]:
with g.as_default():
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # This should print the variable's initial value.
    print v.eval()

    assignment = tf.assign(v, [7])
    # The variable has not been changed yet!
    print v.eval()

    # Execute the assignment op.
    sess.run(assignment)
    # Now the variable is updated.
    print v.eval()

There are many more topics about variables that we didn't cover here, such as loading and storing. To learn more, see the [TensorFlow docs](https://www.tensorflow.org/programmers_guide/variables).

### Exercise #2: Simulate 10 rolls of two dice.

Create a dice simulation, ending up with a `10x3` 2-D tensor in which:

  * Columns `1` and `2` each hold one throw of one dice.
  * Column `3` holds the sum of the two throws on the same row.

For example, the first row might hold something like the following column values:

  * Column `1` holds `4`
  * Column `2` holds `3`
  * Column `3` holds `7`

You'll need to explore the [TensorFlow documentation](https://www.tensorflow.org/api_guides/python/array_ops) to solve this task.

In [ ]:
# Write your code for Task 2 here.